In [ ]:
import re
# from threading import Thread
from multiprocessing import Process
from multiprocessing import Manager
import requests
import time
import xlrd
from bs4 import BeautifulSoup
from lxml import etree
from urllib.parse import quote
from urllib.parse import unquote

In [ ]:
root = 'http://www.webofknowledge.com/'
s = requests.get(root)
sid=re.findall(r'SID=\w+&', s.url)[0].replace('SID=', '').replace('&', '')
print(sid)

In [ ]:
hearders = {
            'Origin': 'https://apps.webofknowledge.com',
            'Referer': 'https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch',
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
            'Content-Type': 'application/x-www-form-urlencoded'
        }
fd = {
            'fieldCount': 3,
            'action': 'search',
            'product': 'WOS',
            'search_mode': 'GeneralSearch',
            'SID': sid,
            'max_field_count': 25,
            'formUpdated': 'true',
            'value(input1)': 'FRONTIERS OF STRUCTURAL AND CIVIL ENGINEERING',
            'value(select1)': 'SO',
            'value(bool_1_2)': 'AND',
            'value(input2)': 'Article### Review',
            'value(select2)': 'DT',
            'value(bool_2_3)': 'AND',
            'value(input3)': '2017-2018',
            'value(select3)': 'PY',  
            'value(hidInput1)': '',
            'value(hidInput2)': '',
            'limitStatus': 'collapsed',
            'ss_lemmatization': 'On',
            'ss_spellchecking': 'Suggest',
            'SinceLastVisit_UTC': '',
            'SinceLastVisit_DATE': '',
            'period': 'Range Selection',
            'range': 'ALL',
            'startYear': '1975',
            'endYear': '2019',
            'update_back2search_link_param': 'yes',
            'ssStatus': 'display:none',
            'ss_showsuggestions': 'ON',
            'ss_query_language': 'auto',
            'ss_numDefaultGeneralSearchFields': 1,
            'rs_sort_by': 'PY.D;LD.D;SO.A;VL.D;PG.A;AU.A'
        }

In [ ]:
root_url = 'https://apps.webofknowledge.com/WOS_GeneralSearch.do'
s = requests.Session()
r = s.post(root_url, data=fd, headers=hearders)
r.encoding = r.apparent_encoding
tree = etree.HTML(r.text)
soup = BeautifulSoup(r.text, 'lxml')
# k=''
# for i in tree.xpath("//div[@class='search-results-content']/div/div/a//text()"):
#     k+=i  
# print(k)
paper_num = int(soup.find('span', attrs={'id': 'footer_formatted_count'}).get_text().replace(',', ''))
print('2016年和2017年文章总数：', paper_num)

In [ ]:
url=soup.find('a', class_='snowplow-citation-report citation-report-summary-link')['href']
wos_url='https://apps.webofknowledge.com'
CR_url=wos_url+url
cr_r = requests.get(CR_url, 3)
cr_r_qid=re.findall(r'qid=\d+&', cr_r.url)[0].replace('qid=', '').replace('&', '')
print('Qid=', cr_r_qid)
cr_r.encoding = cr_r.apparent_encoding
tree2 = etree.HTML(cr_r.text)
soup2 = BeautifulSoup(cr_r.text, 'lxml')

table=soup2.find_all('td', class_='quarter-width')
Total_Publications=table[0].find('em', attrs={'class': 'commafy last'}).get_text().replace(',', '')
h_index = table[1].find('em', attrs={'id': 'H_INDEX'}).get_text().replace(',', '')
Average_citations_per_item=table[1].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')
Sum_of_Times_Cited=table[2].find('em', attrs={'class': 'commafy'}).get_text().replace(',', '')
without_self_citations=table[2].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')
Citing_articles=table[3].find('em', attrs={'class': 'commafy'}).get_text().replace(',', '')
without_self_citations2=table[3].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')

print(Total_Publications, h_index, Average_citations_per_item, Sum_of_Times_Cited, without_self_citations, Citing_articles, without_self_citations2)

CitReportHead=soup2.find('div', class_='CitReportTotalRow1')
CitReportTable=soup2.find('div', class_='CitReportTotalRow2')
CitYear=[]
CitYearValue=[]
for i in CitReportHead.find_all('div', class_='tcPerYear'):
    CitYear.append(i.get_text().strip())
for i in CitReportTable.find_all('div', class_='totalRow'):
    CitYearValue.append(i.get_text().strip())
print(CitYear, '\n',CitYearValue)

In [ ]:
Citing_articles_href=soup2.find('a', class_='margin-left-20 snowplow-cited-rep-analyze-results-citing-articles')['href']
wos_url='https://apps.webofknowledge.com'
Citing_articles_href=wos_url+Citing_articles_href
qid = re.findall(r'qid=\d+&', Citing_articles_href)[0].replace('qid=', '').replace('&', '')
qid=int(qid)
print('Qid=', qid)

r3 = requests.get(Citing_articles_href, 3)
r3.encoding = r3.apparent_encoding
tree3 = etree.HTML(r3.text)
soup3 = BeautifulSoup(r3.text, 'lxml')

In [ ]:
headers2 = {
    'host':'apps.webofknowledge.com',
    'Origin': 'https://wcs.webofknowledge.com',
    'Referer': 'https://wcs.webofknowledge.com/RA/analyze.do?product=WOS&SID='+sid+'&field=PY_PublicationYear_PublicationYear_en&yearSort=true',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    'Content-Type': 'application/x-www-form-urlencoded'
        }
fd2 = {
    'SID': sid,
    'HISTORY':'<b>DOCUMENT TYPES:</b> (ARTICLE OR REVIEW) AND <b>PUBLICATION YEARS:</b> (2018 OR 2017)',
    'queryString':'Total Citing Articles:  PUBLICATION NAME: (frontiers of structural and civil engineering)',
    'action': 'search',
    'product': 'WOS',
    'PRODUCT': 'WOS',
    'canRun':'yes',
    'mode': 'Analyze',
    'viewType':'summary',
    'parentQid': qid+1,
    'resultSetID':qid+1,
    'ua_url':'https://apps.webofknowledge.com',
    'set_base_url':'https://wcs.webofknowledge.com:443/RA',
    'maxSave':100000,
    'queryOption(summary_search_mode)':'TotalCitingArticles',
    'count':173,
    'recordString':173,
    'field':'PY_PublicationYear_PublicationYear_en',
    'recordsStr':'records',
    'ViewRecordsStr':'View Records',
    'visualization':'Bargraph',
    'showNoofResults':25,
    'maxRows':25,
    'refineSelection':'PublicationYear_2019',
        }

In [ ]:
s = requests.Session()
url='https://apps.webofknowledge.com/InboundService.do'
r4 = s.post(url, data=fd2, headers=headers2)
r4.encoding = r4.apparent_encoding
tree4 = etree.HTML(r4.text)
soup4 = BeautifulSoup(r4.text, 'lxml')

paper_num = int(soup4.find('span', attrs={'id': 'footer_formatted_count'}).get_text().replace(',', ''))
print(paper_num)

In [ ]:
# NowCit_href=soup4.find('div', class_='create-cite-report').find('a')['href']
# wos_url='https://apps.webofknowledge.com'
# NowCit_url=wos_url+NowCit_href
# print(NowCit_url)
# r5 = requests.get(Citing_articles_href, 3)
# r5.encoding = r5.apparent_encoding
# tree5 = etree.HTML(r5.text)
# soup5 = BeautifulSoup(r5.text, 'lxml')

In [ ]:
rurl='https://apps.webofknowledge.com/summary.do?mode=Analyze&product=WOS&product=WOS&search_mode=Analyze&parentQid='+str(qid+1)+'&qid='+str(qid+2)+'&SID='+sid+'&SID='+sid

headers3 = {
    'Host':'apps.webofknowledge.com',
    'Origin': 'http://apps.webofknowledge.com',
    'Referer': 'http://apps.webofknowledge.com/InboundService.do?action=search&exclude=',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    'Content-Type': 'application/x-www-form-urlencoded'
        }
fd3 = {
    'viewType':'summary',
    'product':'WOS',
    'rurl':quote(rurl, 'utf-8'),
    'mark_id':'WOS',
    'colName':'WOS',
    'search_mode':'Analyze',
    'locale':'en_US',
    'view_name':'WOS-Analyze-summary',
    'sortBy':'LC.A;PY.D;AU.A.en;SO.A.en;VL.D;PG.A',
    'mode':'OpenOutputService',
    'qid':qid+2,
    'SID':sid,
    'format':'saveToFile',
    'filters':'HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS',
    'mark_to':110,
    'mark_from':1,
    'queryNatural':'<b>PUBLICATION NAME:</b> (frontiers of structural and civil engineering)',
    'count_new_items_marked':0,
    'use_two_ets':'false',
    'IncitesEntitled':'yes',
    'value(record_select_type)':'range',
    'markFrom':1,
    'markTo':110,
    'fields_selection':'HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS',
    'save_options':'tabWinUTF8',
        }

In [ ]:
s = requests.Session()
url='http://apps.webofknowledge.com/OutboundService.do?action=go&&'
r5 = s.post(url, data=fd3, headers=headers3,allow_redirects=False)
Download_url=r5.headers['Location']
print(Download_url)

headers4 = {
    'host':'ets.webofknowledge.com',
    'Referer': 'http://apps.webofknowledge.com/InboundService.do?action=search&exclude=',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
        }
txt=requests.get(Download_url, headers=headers4)
with open('2.txt', 'wb') as f:
    f.write(txt.content)

In [ ]:
rurl='https://apps.webofknowledge.com/summary.do?mode=Analyze&product=WOS&product=WOS&search_mode=Analyze&parentQid='+str(qid+1)+'&qid='+str(qid+2)+'&SID='+sid+'&SID='+sid

headers4 = {
    'host':'ets.webofknowledge.com',
    'Referer': 'http://apps.webofknowledge.com/InboundService.do?action=search&exclude=',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
        }
fd4 = {
    'mark_to':110,
    'mark_from':1,
    'product':'UA',
    'displayUsageInfo':'true',
    'parentQid':qid+2,
    'rurl':quote(rurl, 'utf-8'),
    'colName':'WOS',
    'filters':'HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS',
    'sortBy':'LC.A;PY.D;AU.A.en;SO.A.en;VL.D;PG.A',
    'qid':qid+3,
    'SID':sid,
    'totalMarked':110,
    'action':'saveToFile',
    'logEventQid':qid+2,
    'displayTimesCited':'true',
    'displayCitedRefs':'true',
    'fileOpt':'tabWinUTF8',
    'UserIDForSaveToRID':'null'
        }

In [ ]:
num=110
url1='http://ets.webofknowledge.com/ETS/ets.do?mark_from=1&product=UA&colName=WOS&displayUsageInfo=true&parentQid='+str(qid+2)+'&rurl='
url2='http://apps.webofknowledge.com/summary.do?mode=Analyze&product=WOS&product=WOS&search_mode=Analyze&parentQid='+str(qid+1)+'&qid='+str(qid+2)+'&SID='+sid+'&SID='+sid
url3='&mark_to='+str(num)+'&filters='
url4='HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS  '
url5='&qid='+str(qid+3)+'&SID='+sid+'&totalMarked='+str(num)+'&action=saveToFile&sortBy=PY.D;LD.D;SO.A;VL.D;PG.A;AU.A&logEventQid='+str(qid+2)+'&displayTimesCited=true&displayCitedRefs=true&fileOpt=tabWinUTF8&UserIDForSaveToRID=null'
url2=quote(url2, 'utf-8')
Download_url=url1+quote(url2, 'utf-8')+url3+quote(url4, 'utf-8')+url5
print(Download_url)
txt=requests.get(Download_url, headers=headers4)
print(txt.url)
with open('2.txt', 'wb') as f:
    f.write(txt.content)

In [ ]:
root_url = 'http://apps.webofknowledge.com/WOS_AdvancedSearch.do'
s = requests.Session()
r = s.post(root_url, data=fd, headers=hearders)
r.encoding = r.apparent_encoding
with open('1.txt', 'wb') as f:
    f.write(r.content)

In [ ]:
z='http://ets.webofknowledge.com/ETS/ets.do?mark_from=1&product=UA&colName=WOS&displayUsageInfo=true&parentQid=2&rurl=http%253A%252F%252Fapps.webofknowledge.com%252Fsummary.do%253Fproduct%253DWOS%2526product%253DWOS%2526search_mode%253DGeneralSearch%2526search_mode%253DGeneralSearch%2526qid%253D2%2526SID%253D7CKYyHdjDrWgJOg9lMI%2526SID%253D7CKYyHdjDrWgJOg9lMI&mark_to=100&filters=HIGHLY_CITED%20HOT_PAPER%20OPEN_ACCESS%20PMID%20USAGEIND%20AUTHORSIDENTIFIERS%20ACCESSION_NUM%20FUNDING%20SUBJECT_CATEGORY%20JCR_CATEGORY%20LANG%20IDS%20PAGEC%20SABBR%20CITREFC%20ISSN%20PUBINFO%20KEYWORDS%20CITTIMES%20ADDRS%20CONFERENCE_SPONSORS%20DOCTYPE%20ABSTRACT%20CONFERENCE_INFO%20SOURCE%20TITLE%20AUTHORS%20%20&qid=4&SID=7CKYyHdjDrWgJOg9lMI&totalMarked=100&action=saveToFile&sortBy=PY.D;LD.D;SO.A;VL.D;PG.A;AU.A&logEventQid=2&displayTimesCited=true&displayCitedRefs=true&fileOpt=tabWinUTF8&UserIDForSaveToRID=null'
z=unquote(z)
print(unquote(z))